In [1]:
import numpy as np
from scipy import sparse

## exact diagonalization

In [2]:
from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator
from quspin.tools.lanczos import lanczos_full,lanczos_iter,FTLM_static_iteration,LTLM_static_iteration
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [3]:
Lx, Ly = 4, 4 # linear dimension of spin 1 2d lattice
N_2d = Lx*Ly # number of sites for spin 1
s = np.arange(N_2d) # sites [0,1,2,....]
x = s%Lx # x positions for sites
y = s//Lx # y positions for sites
T_x = (x+1)%Lx + Lx*y # translation along x-direction
T_y = x +Lx*((y+1)%Ly) # translation along y-direction
P_x = x + Lx*(Ly-y-1) # reflection about x-axis
P_y = (Lx-x-1) + Lx*y # reflection about y-axis
Z   = -(s+1) # spin inversion
# basis_2d = spin_basis_general(N_2d,pauli=False)
spin_basis_general(N_2d,kxblock=(T_x,0),kyblock=(T_y,0),pxblock=(P_x,0),pyblock=(P_y,0),zblock=(Z,0), pauli=True)

# set up coupling 
J1 = 1
J2 = 1
J_zz = [[J1,i,T_x[i]] for i in range(N_2d)]+[[J1,i,T_y[i]] for i in range(N_2d)] 
J_zz = J_zz + [[J2,0,5],[J2,2,7],[J2,8,13],[J2,10,15],[J2,1,14],[J2,3,12],[J2,6,9],[J2,4,11]]
J_xy = [[J1/2.0,i,T_x[i]] for i in range(N_2d)]+[[J1/2.0,i,T_y[i]] for i in range(N_2d)]
J_xy = J_xy + [[J2/2.0,0,5],[J2/2.0,2,7],[J2/2.0,8,13],[J2/2.0,10,15],[J2/2.0,1,14],[J2/2.0,3,12],[J2/2.0,6,9],[J2/2.0,4,11]]
ops_dict = dict(Jpm=[["+-",J_xy]],Jmp=[["-+",J_xy]],Jzz=[["zz",J_zz]])

/Users/keisukemurota/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:12: GeneralBasisWarning: using non-commuting symmetries can lead to unwanted behaviour of general basis, make sure that quantum numbers are invariant under non-commuting symmetries!
  if sys.path[0] == '':


In [4]:
E = []
for i in range(Lx*Ly+1):
    print(i)
    basis_2d = spin_basis_general(N_2d, Nup = i ,pauli=False)
    H_ = quantum_operator(ops_dict,basis=basis_2d,dtype=np.float64, check_symm=False)
    E_,V_ = H_.eigh({})
    E.append(E_)
E = np.concatenate(E,axis=0)
T = 1
beta = 1/T
# del H2D
np.sum(E * np.exp(-beta*E))/np.sum(np.exp(-beta*E)) / 16

0
Hermiticity check passed!
Particle conservation check passed!
1
Hermiticity check passed!
Particle conservation check passed!
2
Hermiticity check passed!
Particle conservation check passed!
3
Hermiticity check passed!
Particle conservation check passed!
4
Hermiticity check passed!
Particle conservation check passed!


KeyboardInterrupt: 

In [13]:
np.sum(E * np.exp(-beta*E))/np.sum(np.exp(-beta*E))

-5.585519722674071

## apply unitary transformation to Shastry-Sutherland

local state will be transformed from standard basis to singlet & triplet.

### original local hamiltonian

In [5]:
I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

h1 = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h2 = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real




h1_ = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real

h2_ = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real



h = h1 + h2

h_ = h1_ + h2_


### construct unitary transformation

In [6]:
u = np.array([
    [0,1,0,0],
    [1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [-1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [0,0,0,1]
])
u = sparse.csr_matrix(u)
U = sparse.kron(u, u,format='csr')



In [7]:
H = U.T @ h @ U
H2 = U.T @ h_ @ U


np.save("array/H.npy",H.toarray())
np.save("array/H2.npy",H2.toarray())

In [22]:
print(H)

  (9, 4)	-0.4999999999999999
  (6, 4)	-0.4999999999999999
  (5, 5)	-0.5
  (9, 6)	0.4999999999999999
  (4, 6)	-0.4999999999999999
  (7, 7)	0.5
  (8, 7)	0.4999999999999999
  (10, 7)	0.4999999999999999
  (13, 8)	-0.4999999999999999
  (7, 8)	0.4999999999999999
  (4, 9)	-0.4999999999999999
  (6, 9)	0.4999999999999999
  (13, 10)	0.4999999999999999
  (7, 10)	0.4999999999999999
  (12, 11)	0.4999999999999999
  (14, 11)	0.4999999999999999
  (11, 12)	0.4999999999999999
  (14, 12)	0.4999999999999999
  (8, 13)	-0.4999999999999999
  (10, 13)	0.4999999999999999
  (13, 13)	0.5
  (11, 14)	0.4999999999999999
  (12, 14)	0.4999999999999999
  (15, 15)	-0.5
